In [8]:
import time
import json
import pandas as pd
import numpy as np

In [7]:
# PATHs

## The directory where the dune data is stored
DUNE_DATA_PATH = '../dune_data'

## The directory where the cryptopunk data with attributes info is stored
CSV_PATH = '../cp'

## The directory where the three databases are stored
DATABASE_PATH = '../database'

## The directory where the cryptopunk images is stored
PUNK_IMG_PATH='../punk_imgs'

## The directory where the visualization data for each visualization is stored
VIS_DATA_PATH = '../vis_data'

## The directory where the scraped tweets data is stored
TWEET_PATH = '../tweets'

In [15]:
# read transaction database
tx_db = pd.read_csv('{}/tx_db.csv'.format(DATABASE_PATH), index_col=0)

# read cryptopunk database
punk_db = pd.read_csv('{}/punk_db.csv'.format(DATABASE_PATH), index_col=0)
punk_db['attributes'] = punk_db['attributes'].apply(eval)

# read trader database
trader_db = pd.read_csv('{}/trader_db.csv'.format(DATABASE_PATH), index_col=0)


### 3. Bubble Plot

<img src="../imgs/vis3_example.png" alt="drawing" width="500">

Data format

```
Index
0   tx: a single transaction

Column
1   tx_id
2   date
3   price
4   bubble_size: the attribute count of the punk
5   bubble_color: decided by the skin tone of the punk
6   from: trader_id
7   to: trader_id
8   punk_id
9   punk_img: the image directory of the punk
```

In [16]:
def set_bubble_color(x):
    if x == 'Dark':
        return '#A4031F'
    elif x == 'Medium':
        return '#DB9065'
    elif x == 'Light':
        return '#F2A359'
    else:
        return '#F2DC5D'


def get_dataset_vis3(tx_db, punk_db, trader_db):
    vis3_df = tx_db.copy(deep=True)

    # drop the punk transfer transactions
    vis3_df.dropna(axis=0, inplace=True)

    # set date
    vis3_df['date'] = vis3_df['date_time']  # .apply(lambda x: x.split(' ')[0])

    # set punk_skin_color as bubble color
    vis3_df['punk_skin_color'] = vis3_df['punk_id'].apply(
        lambda x: punk_db.loc[x, 'skin_tone'])
    vis3_df['bubble_color'] = vis3_df['punk_skin_color'].apply(
        set_bubble_color)

    # set punk_attr_count as bubble size
    vis3_df['punk_attr_count'] = vis3_df['punk_id'].apply(
        lambda x: punk_db.loc[x, 'attr_count'])
    vis3_df.rename(
        columns={'punk_attr_count': 'bubble_size', 'eth_price': 'price'}, inplace=True)

    # get punk image directory
    # vis3_df['punk_img'] = vis3_df['punk_id'].apply(
    #     lambda x: punk_db.loc[x, 'img_dir'])
    # vis3_df['punk_img'] = vis3_df['punk_id'].apply(lambda x: punk_db.loc[x, 'img_dir'][1:])

    vis3_df['img_url'] = vis3_df['punk_id'].apply(lambda x: punk_db.loc[x, 'img_url'])

    return vis3_df.loc[:, ['date',  'price', 'bubble_size', 'bubble_color', 'from', 'to', 'punk_id', 'img_url']]


vis3_df = get_dataset_vis3(tx_db, punk_db, trader_db)
vis3_df.to_csv('{}/vis3_data.csv'.format(VIS_DATA_PATH))
vis3_df


,date,price,bubble_size,bubble_color,from,to,punk_id,img_url
tx_id,,,,,,,,
0,2017-06-23 21:06:32,0.01,3,#DB9065,4056,459,3134,https://www.larvalabs.com/cryptopunks/cryptopu...
1,2017-06-23 21:06:45,0.04,3,#DB9065,4056,4600,5719,https://www.larvalabs.com/cryptopunks/cryptopu...
2,2017-06-23 21:06:53,0.10,4,#F2DC5D,459,2855,5056,https://www.larvalabs.com/cryptopunks/cryptopu...
3,2017-06-23 22:06:08,0.06,3,#F2A359,4056,4600,5624,https://www.larvalabs.com/cryptopunks/cryptopu...
4,2017-06-23 22:06:12,0.03,4,#F2DC5D,4056,4600,6548,https://www.larvalabs.com/cryptopunks/cryptopu...
...,...,...,...,...,...,...,...,...
16817,2022-05-04 20:05:55,61.50,3,#DB9065,2283,3447,1648,https://www.larvalabs.com/cryptopunks/cryptopu...
16818,2022-05-05 05:05:38,66.66,3,#F2DC5D,687,5505,2625,https://www.larvalabs.com/cryptopunks/cryptopu...
16819,2022-05-05 21:05:57,70.00,2,#DB9065,4083,789,3314,https://www.larvalabs.com/cryptopunks/cryptopu...


### Vis3.2 3D Scatter Plot

In [17]:
punk_db

,type,gender,skin_tone,attr_count,attributes,img_dir,avg_price,img_url
punk_id,,,,,,,,
0,Human,Female,Medium,3,"[Green Eye Shadow, Earring, Blonde Bob]",../punk_imgs/0.png,0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
1,Human,Male,Dark,2,"[Smile, Mohawk]",../punk_imgs/1.png,31.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
2,Human,Female,Light,1,[Wild Hair],../punk_imgs/2.png,0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
3,Human,Male,Dark,3,"[Wild Hair, Nerd Glasses, Pipe]",../punk_imgs/3.png,0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
4,Human,Male,Medium,4,"[Big Shades, Wild Hair, Earring, Goat]",../punk_imgs/4.png,0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
...,...,...,...,...,...,...,...,...
9995,Human,Female,Albino,2,"[Purple Eye Shadow, Straight Hair Dark]",../punk_imgs/9995.png,0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
9996,Human,Male,Light,4,"[Cigarette, Earring, Crazy Hair, Smile]",../punk_imgs/9996.png,0.000000,https://www.larvalabs.com/cryptopunks/cryptopu...
9997,Zombie,Male,NaN,2,"[Front Beard, Cap Forward]",../punk_imgs/9997.png,99.990000,https://www.larvalabs.com/cryptopunks/cryptopu...


In [25]:
def get_dataset_vis3_2(tx_db, punk_db, trader_db):
    df = punk_db.loc[:, ['type', 'gender', 'skin_tone', 'attr_count', 'img_url', 'avg_price']]
    df = df[df['avg_price'] != 0]
    df['category'] = df.apply(lambda x: x['type']+' '+x['gender'], axis=1)
    df['bubble_color'] = df['skin_tone'].apply(set_bubble_color)
    df['skin_tone'] = df['skin_tone'].apply(lambda x: 'Non-human' if pd.isna(x) else x)
    return df

vis3_2_df = get_dataset_vis3_2(tx_db, punk_db, trader_db)
vis3_2_df.to_csv('{}/vis3_2_data.csv'.format(VIS_DATA_PATH))
vis3_2_df

,type,gender,skin_tone,attr_count,img_url,avg_price,category,bubble_color
punk_id,,,,,,,,
1,Human,Male,Dark,2,https://www.larvalabs.com/cryptopunks/cryptopu...,31.000000,Human Male,#A4031F
14,Human,Female,Dark,3,https://www.larvalabs.com/cryptopunks/cryptopu...,1.750000,Human Female,#A4031F
53,Human,Female,Light,2,https://www.larvalabs.com/cryptopunks/cryptopu...,57.330000,Human Female,#F2A359
89,Human,Female,Dark,1,https://www.larvalabs.com/cryptopunks/cryptopu...,19.990000,Human Female,#A4031F
116,Human,Female,Dark,4,https://www.larvalabs.com/cryptopunks/cryptopu...,1.800000,Human Female,#A4031F
...,...,...,...,...,...,...,...,...
9980,Human,Female,Medium,4,https://www.larvalabs.com/cryptopunks/cryptopu...,33.815000,Human Female,#DB9065
9982,Human,Female,Light,3,https://www.larvalabs.com/cryptopunks/cryptopu...,142.000000,Human Female,#F2A359
9990,Human,Male,Albino,2,https://www.larvalabs.com/cryptopunks/cryptopu...,36.150003,Human Male,#F2DC5D
